In [ ]:
pip install autogen-agentchat~=0.2 autogen apify-client

In [ ]:
pip install pyautogen spider-client

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 3.8 MB/s eta 0:00:00
  Created wheel for spider-client: filename=spider_client-0.1.26-py3-none-any.whl size=13032 sha256=630fbca0696fa1ddad9850bd76c401783c43e614d73d9ad045a97ec8de7014af
  Stored in directory: /root/.cache/pip/wheels/72/af/3b/0a1028d06412a2c275abecfc222e1dbb58832144eb3389fb0e
Successfully built spider-client


In [ ]:
import autogen

# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
#     },
# )
config_list = [{"model": "gpt-4o", "api_key": "%%%%%%"}]
apify_api_key = "********"
spider_api_key = "*****"

In [ ]:
from apify_client import ApifyClient
from typing_extensions import Annotated

In [ ]:
from typing import Any, Dict, List

from spider import Spider
from typing_extensions import Annotated


def scrape_page(
    url: Annotated[str, "The URL of the web page to scrape"],
    params: Annotated[dict, "Dictionary of additional params."] = None,
) -> Annotated[Dict[str, Any], "Scraped content"]:
    # Initialize the Spider client with your API key, if no api key is specified it looks for SPIDER_API_KEY in your environment variables
    client = Spider(spider_api_key)

    if params is None:
        params = {"return_format": "markdown"}

    scraped_data = client.scrape_url(url, params)
    return scraped_data[0]


def crawl_page(
    url: Annotated[str, "The url of the domain to be crawled"],
    params: Annotated[dict, "Dictionary of additional params."] = None,
) -> Annotated[List[Dict[str, Any]], "Scraped content"]:
    # Initialize the Spider client with your API key, if no api key is specified it looks for SPIDER_API_KEY in your environment variables
    client = Spider(spider_api_key)

    if params is None:
        params = {"return_format": "markdown"}

    crawled_data = client.crawl_url(url, params)
    return crawled_data

In [ ]:
from autogen import ConversableAgent, register_function

# Create web scraper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config={"config_list": config_list},
    system_message="You are a web scraper and you can scrape any web page to retrieve its contents."
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create web crawler agent.
crawler_agent = ConversableAgent(
    "WebCrawler",
    llm_config={"config_list": config_list},
    system_message="You are a web crawler and you can crawl any page with deeper crawling following subpages."
    "Returns 'TERMINATE' when the scraping is done.",
)


In [ ]:
# Create user proxy agent.
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 4,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)

llm_config = {"config_list": config_list, "cache_seed": 42}
conso = autogen.AssistantAgent(
    name="Consolidator",
    system_message="summarize and consolidate infomration and provide personalised recommendations",
    llm_config=llm_config,
)

# Register the functions with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

register_function(
    crawl_page,
    caller=crawler_agent,
    executor=user_proxy,
    name="crawl_page",
    description="Crawl an entire domain, following subpages and return the content.",
)

In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, crawler_agent, scraper_agent, conso], messages=[], max_round=10)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Recommend me suitable insurance plans which Manulige offers (https://www.manulife.com.hk/en/individual/products.html) base on my profile. I am the only child of my family who age 35, working as a data scientist with salary of 70K HKD per month. My parents are 70 years old and living with me. Our total asset for the family is around 2 million. Recommend me the suitable insurance plans from Manulife and only Manulife but not other insurance firms"
)

User_proxy (to chat_manager):

Recommend me suitable insurance plans which Manulige offers (https://www.manulife.com.hk/en/individual/products.html) base on my profile. I am the only child of my family who age 35, working as a data scientist with salary of 70K HKD per month. My parents are 70 years old and living with me. Our total asset for the family is around 2 million. Recommend me the suitable insurance plans from Manulife and only Manulife but not other insurance firms

--------------------------------------------------------------------------------

Next speaker: WebScraper


>>>>>>>> USING AUTO REPLY...
WebScraper (to chat_manager):

***** Suggested tool call (call_nIRXNhxRYktgUGSlfUcjrnMD): scrape_page *****
Arguments: 
{"url":"https://www.manulife.com.hk/en/individual/products.html"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: User_proxy


>>>>>>>>

ChatResult(chat_id=None, chat_history=[{'content': 'Recommend me suitable insurance plans which Manulige offers (https://www.manulife.com.hk/en/individual/products.html) base on my profile. I am the only child of my family who age 35, working as a data scientist with salary of 70K HKD per month. My parents are 70 years old and living with me. Our total asset for the family is around 2 million. Recommend me the suitable insurance plans from Manulife and only Manulife but not other insurance firms', 'role': 'assistant', 'name': 'User_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_nIRXNhxRYktgUGSlfUcjrnMD', 'function': {'arguments': '{"url":"https://www.manulife.com.hk/en/individual/products.html"}', 'name': 'scrape_page'}, 'type': 'function'}], 'name': 'WebScraper', 'role': 'assistant'}, {'content': 'Error: list index out of range', 'tool_responses': [{'tool_call_id': 'call_nIRXNhxRYktgUGSlfUcjrnMD', 'role': 'tool', 'content': 'Error: list index out of range'}], 'role': 'tool',